In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

### 1. Phoneme Dataset
Load the phoneme dataset using Pandas

In [ ]:
df = pd.read_csv('data/phoneme.csv')
# print(data)

# For the speaker variable, as well as the phoneme variable `g`, we can make use of the pandas `category` data type
df = df.astype({'g':'category','speaker':'category'})

X = np.array(df.drop(['row.names', 'g', 'speaker'], axis=1))

speaker = df.speaker

# Split the dataset into a train and test dataset according to column "speaker".
# Be sure to exclude row number, "speaker" and response columns from your features.

cat_names = df.g.cat.categories
y = df.g.cat.codes

y = np.array(y)
X = np.array(X)
print(X.shape)

In [ ]:
def split_by(X,y,tst_by):
    from types import SimpleNamespace
    return SimpleNamespace(x_tst=X[tst_by],y_tst=y[tst_by],
                           x_trn=X[~tst_by],y_trn=y[~tst_by])

data = split_by(X,y,tst_by=df.speaker.str.startswith('test'))

### 2. LDA modelling
Fit an LDA model. Compute and report the train and test error.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

def fit_discriminant_analysis(data, cls=LinearDiscriminantAnalysis,quiet=False):
    from types import SimpleNamespace
    clf = cls()
    clf.fit(data.x_trn, data.y_trn)
    def get_model_acc(X, y):
        pred = clf.predict(X)
        return np.mean(pred == y)
    acc_trn = get_model_acc(data.x_trn,data.y_trn)
    acc_tst = get_model_acc(data.x_tst,data.y_tst)
    if not quiet:
        print(f'The accuracy of the model is:\nTrain:\t{acc_trn*100:.4f}%\nTest:\t{acc_tst*100:.4f}%')
    return clf,SimpleNamespace(trn=acc_trn, tst=acc_tst)
clf_lda6,acc_lda6 = fit_discriminant_analysis(data)


### 3. LDA canonical coordinates
 Plot the projection of the training data onto the first two canonical coordinates of the LDA and report your findings. Investigate the data projected on further dimensions.

In [ ]:
def project_lda_single(X,y,W,dim1,dim2,legend=True,colorbar=True):
    xproj = X @ W[[dim1-1,dim2-1],:].T
    colors = ['red','green','blue','purple', 'yellow']
    hnd = plt.scatter(xproj[:,0], xproj[:,1], c = y, cmap = matplotlib.colors.ListedColormap(colors), marker='x')
    plt.title('Projection on canonical coordinates')
    plt.xlabel(f'Coordinate {dim1}')
    plt.ylabel(f'Coordinate {dim2}')
    ax = plt.gca()
    if legend:
        plt.legend(hnd.legend_elements()[0],cat_names, title="Classes")
    if colorbar:
        plt.colorbar()

def project_lda(X,y,clf,dims):
    n = len(dims)
    W = clf.coef_
    for i,(dim1,dim2) in enumerate(dims):
        plt.subplot(1,n,i+1)
        project_lda_single(X,y,W,dim1,dim2)
    
plt.figure(figsize=(15,5))
project_lda(data.x_trn,data.y_trn,clf_lda6,((1,2),(1,3),(4,5)))

In [ ]:
def project_all(X,y,clf):
    W = clf.coef_
    n = W.shape[0]
    for i in range(n):
        for j in range(n):
            plt.subplot(n,n,n*j+i+1)
            if i==j:
                plt.text(0,0,f'Coordinate {i+1}',
                         horizontalalignment='center',verticalalignment='center',size=20)
                plt.xlim(-1,1)
                plt.ylim(-1,1)
            else:
                project_lda_single(X,y,W,i+1,j+1,legend=False,colorbar=False)
                plt.xlabel(None);plt.ylabel(None);plt.title(None)
plt.figure(figsize=(15,15))
project_all(X,y,clf_lda6)

We  see that the canonical directions can be used to differentiate the classes.
The first coordinate can differentiate the two class categories, consisting of (blue,purple,yellow) and the (red,green) ones, by adding a linear threshold at a value of about 40.
Adding the second canonical coordiante we can additionally classify well the red and green classes or the blue and yellow ones, by using a diagonal line along these two directions. The process goes on, where we can further linearly separate more classes, as more directions are available.

This also reveals the capacity of LDA to be used as a supervised dimensionality reduction method, which we will (probably) discuss in future lectures.

### 4. LDA on "aa", "ao"
Select the two phonemes "aa" and "ao". Fit an LDA model on this data set and repeat the steps
    done in (2).

In [ ]:
def filter_y(data, keep_only):
    from types import SimpleNamespace
    idx_tst = np.isin(data.y_tst,keep_only)
    idx_trn = np.isin(data.y_trn,keep_only)
    return SimpleNamespace(x_tst=data.x_tst[idx_tst,:],y_tst=data.y_tst[idx_tst],
                           x_trn=data.x_trn[idx_trn,:],y_trn=data.y_trn[idx_trn])
    


idl_kept = np.where(cat_names.isin(['aa','ao']))[0]
cat_names2 = cat_names[idl_kept]
data2 = filter_y(data, idl_kept)

In [ ]:
clf_lda2,acc_lda2 = fit_discriminant_analysis(data2)

## 5. QDA
Repeat steps (b) and (d) using QDA and report your findings. Would you prefer LDA or QDA in this example? Why?

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf_qda2,acc_qda2 = fit_discriminant_analysis(data2,cls=QuadraticDiscriminantAnalysis)

## 6. Confusion Matrices
Generate confusion matrices for the LDA and QDA model for "aa" and "ao". Which differences can you observe between the models?

In [ ]:
def show_cm(X,y,clf, nrm=False):
    cm = confusion_matrix(y, clf.predict(X))
    if nrm:
        cm = cm/cm.sum()*100
    cm_display = ConfusionMatrixDisplay(cm).plot(ax=plt.gca())
    plt.xticks([0,1],cat_names2)
    plt.yticks([0,1],cat_names2)

def show_data_cm(data, clf, nrm=False):
    plt.figure(figsize=(14,5))
    plt.subplot(1,2,1)
    plt.title('Train')
    show_cm(data.x_trn,data.y_trn,clf,nrm=nrm)
    plt.subplot(1,2,2)
    plt.title('Test')
    show_cm(data.x_tst,data.y_tst,clf,nrm=nrm)

For LDA we compute the confusion matrices

In [ ]:
show_data_cm(data2, clf_lda2)

For QDA they become

In [ ]:
show_data_cm(data2, clf_qda2)

In this example we can see that QDA is clearly overfitting the two phonemes, since its training accuracy is 100%, but the testing accuracy is very low.

In addition, the predictor opts much more in favour of the most likely class, which is class 1, and the errors are not balanced between the two classes.

Therefore, the simpler model of LDA is preferred.

## Splitting the data<sup>1</sup>

This exercise was a good chance to investigate the effect of splitting the data.

Here we first load the information of each speaker, and then give a framework to experiment with stratified splits and see their effects.

<sup>1</sup><small>This part was not required for the full points of this problem</small>

In [ ]:
def get_speaker_info():
    import requests
    from io import StringIO
    import re
    lines = requests.get('https://catalog.ldc.upenn.edu/docs/LDC93S1/SPKRINFO.TXT').text
    s = pd.Series(lines.split('\n')[:-1])
    idl_cmt = s.str.startswith(';')
    hdr = re.split(r'\s+', s[idl_cmt].iloc[-2][1:])
    
    df_sp = s[~idl_cmt].str.split(r'\s+',n=9,expand=True)
    df_sp.columns = hdr
    return df_sp
df_sp_inf = get_speaker_info()
df_sp_inf.head()

df_sp = speaker.str.split('.',expand=True).iloc[:,:-1]
df_sp.columns = ['split','dr','id']

df_sp = df_sp.assign(ID=lambda x:x.id.str.slice(1).str.upper())
df_sp_full = pd.merge(df_sp,df_sp_inf,on='ID')

df_sp_full.head()


The full information for each speaker includes:
 * their dialect region
 * education level
 * race
 * name (identity)
 
We now investigate the effect of using different kinds of splits, for the same percentage of test-train sizes in each split as the one given in the original dataset.

In [ ]:
from utilities import split_data
from tqdm import tqdm # For nice progress bars

def split_stratified(X,y,stratify_by,tst_ratio=.2,seed=0):
    from types import SimpleNamespace
    from sklearn.model_selection import StratifiedShuffleSplit
    sss = StratifiedShuffleSplit(n_splits=1,test_size=tst_ratio,random_state=seed)
    idx_trn, idx_tst = next(sss.split(X,stratify_by))
    return SimpleNamespace(x_tst=X[idx_tst],y_tst=y[idx_tst],by_trn=stratify_by[idx_trn],idx_trn=idx_trn,
                            x_trn=X[idx_trn],y_trn=y[idx_trn],by_tst=stratify_by[idx_tst],idx_tst=idx_tst)


def test_split_fn(fn_split,cls=LinearDiscriminantAnalysis,seed=0, k=1):
    accs = []
    for seed in tqdm(range(k)):
        data = fn_split(seed)
        data2 = filter_y(data, np.where(cat_names.isin(['aa','ao'])))
        clf,acc = fit_discriminant_analysis(data2, cls=cls, quiet=True)
        accs.append(tuple(acc.__dict__.values()))
    acc_trn, acc_tst = zip(*accs)
    return acc_trn, acc_tst

def plot_accuracies_by_split(accs, accs_orig, tag):
    x = np.arange(5)+1
    meths = ['random','by speaker','by region','by education','by race']
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.boxplot([accs.rnd[0],accs.by_id[0],accs.by_dr[0],accs.by_edu[0],accs.by_race[0]])
    plt.ylabel('Accuracy')
    plt.title(f'Training set {tag}')
    plt.xticks(x, meths)
    plt.axhline(accs_orig.trn)
    plt.subplot(1,2,2)
    plt.boxplot([accs.rnd[1],accs.by_id[1],accs.by_dr[1],accs.by_edu[1],accs.by_race[1]])
    plt.xticks(x, meths)
    plt.axhline(accs_orig.tst)
    plt.title(f'Testing set {tag}')
    

In [ ]:
# We perform a training and classification operation for both LDA and QDA and for each method to split the data,
# and record the accuracies for 50 iterations
tst_ratio=len(data.x_tst)/y.shape[0]
k = 50
from types import SimpleNamespace
accs_lda = SimpleNamespace(
    rnd = test_split_fn(lambda seed:split_data(X,y,tst_ratio=tst_ratio,seed=seed),k=k),
    by_id = test_split_fn(lambda seed:split_stratified(X,y,stratify_by=df_sp_full.ID, tst_ratio=tst_ratio, seed=seed),k=k),
    by_dr = test_split_fn(lambda seed:split_stratified(X,y,stratify_by=df_sp_full.DR, tst_ratio=tst_ratio, seed=seed),k=k),
    by_edu = test_split_fn(lambda seed:split_stratified(X,y,stratify_by=df_sp_full.Edu, tst_ratio=tst_ratio, seed=seed),k=k),
    by_race = test_split_fn(lambda seed:split_stratified(X,y,stratify_by=df_sp_full.Race, tst_ratio=tst_ratio, seed=seed),k=k)
)
    

In [ ]:
# Here the same for QDA
k = 50
cls = QuadraticDiscriminantAnalysis
accs_qda = SimpleNamespace(
    rnd = test_split_fn(lambda seed:split_data(X,y,tst_ratio=tst_ratio,seed=seed),cls=cls,k=k),
    by_id = test_split_fn(lambda seed:split_stratified(X,y,stratify_by=df_sp_full.ID, tst_ratio=tst_ratio, seed=seed),cls=cls,k=k),
    by_dr = test_split_fn(lambda seed:split_stratified(X,y,stratify_by=df_sp_full.DR, tst_ratio=tst_ratio, seed=seed),cls=cls,k=k),
    by_edu = test_split_fn(lambda seed:split_stratified(X,y,stratify_by=df_sp_full.Edu, tst_ratio=tst_ratio, seed=seed),cls=cls,k=k),
    by_race = test_split_fn(lambda seed:split_stratified(X,y,stratify_by=df_sp_full.Race, tst_ratio=tst_ratio, seed=seed),cls=cls,k=k)
)


We now create boxplots for the accuracy of each method.
The horizontal line is the corresponding accuracty of the orignal split, as the one given in the data.

We see that for LDA we can do much better by performing a good split, that is better than the one in the authors in almost all cases. For QDA this is the opposite case.

Although these experiments do not provide enough evidence for definitive answers, the stratified split based on the race seems promising, and so does the region variable.


In [ ]:
plot_accuracies_by_split(accs_lda, acc_lda2, '- LDA')
plot_accuracies_by_split(accs_qda, acc_qda2, '- QDA')
